<a href="https://colab.research.google.com/github/Harrow-Enigma/spring-2022/blob/main/MNIST-demo/Handwritten_Digits_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Enigma Logo" height="100px" src="https://avatars.githubusercontent.com/u/74505663?v=4" align="left" hspace="10px" vspace="0px"></p>

# Handwritten Digits Recognition
*By Team Enigma*

Based off [this official PyTorch tutorial](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html).

In [ ]:
#@title Copyright 2022 Team Enigma, licensed under the GNU GPL v3 License

print("""
MNIST Handwritten Digits Recognition with Machine Learning.
Copyright (C) 2022  Team Enigma

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
""")

## Data Loading

In [ ]:
#@title Importing Libraries and Getting Datasets
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

MNIST_TRAIN = datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
MNIST_TEST = datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(MNIST_TRAIN, batch_size=64, shuffle=True)
test_dataloader = DataLoader(MNIST_TEST, batch_size=64, shuffle=True)

In [ ]:
#@title Data Visualisation
sample_index =  3#@param {type:"integer"}

sample_image, sample_label = MNIST_TRAIN[sample_index]
plt.imshow(sample_image[0], cmap="gray")
plt.show()
print(f'Training data index {sample_index}: image of shape {sample_image.shape}, label {sample_label}')

## Creating Neural Network

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(28 * 28, 512),    # fully connected perceptron layer
            nn.ReLU(),                  # ReLU activation function
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.stack(x)
        return logits

model = NeuralNet()

print(model)

## Training

In [ ]:
#@title Training Hyperparameters
LEARNING_RATE = 1e-3 #@param {type:"number"}
EPOCHS = 10 #@param {type:"slider", min:1, max:100, step:1}


In [ ]:
loss_fn = nn.CrossEntropyLoss() # specify loss function
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE) # specify gradient descent algorithm

In [ ]:
def train_loop():
    size = len(train_dataloader.dataset)

    # load batches
    for batch, (inp, targ) in enumerate(train_dataloader):
        
        pred = model(inp)           # model makes predictions
        loss = loss_fn(pred, targ)  # loss calculated

        optimizer.zero_grad()       # clear previous gradients
        loss.backward()             # perform differentiation and calculate gradients
        optimizer.step()            # apply gradient descent!

        # info printing
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(targ)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# testing model
def test_loop():
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for inp, targ in test_dataloader:
            pred = model(inp)
            test_loss += loss_fn(pred, targ).item()
            correct += (pred.argmax(1) == targ).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# confusion matrix
def show_confusion_matrix():
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inp, targ in test_dataloader:
            pred = model(inp)
            y_true.extend(targ.numpy().tolist())
            y_pred.extend(pred.argmax(1).numpy().tolist())
    matrix = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(10, 10))
    sns.heatmap(matrix,
                xticklabels=[i for i in range(10)],
                yticklabels=[i for i in range(10)],
                annot=True, fmt='g')
    plt.xlabel('Prediction')
    plt.ylabel('Label')
    plt.show()

In [ ]:
#@title Model Training!
for ep in range(EPOCHS):
    print(f"Epoch {ep+1} starting")
    train_loop()
    test_loop()

In [ ]:
show_confusion_matrix()

## Prediction

Now go to [Pixilart](https://www.pixilart.com/draw)! Load in the template I've given you, draw a number, and see if the AI model recognises it!

In [ ]:
#@title Image Prediction Tool
#@markdown Enter your image filename in the field below, and we will use your trained AI model to predict the digit!

def load_image(fpath):
    transform = torchvision.transforms.Grayscale()
    img = torchvision.io.read_image(fpath)
    gray_img = transform(img[:3]).type(torch.float32)
    return gray_img

def predict(fpath):
    img = load_image(fpath)
    plt.imshow(img[0], cmap="gray")
    plt.show()

    logits = model(img)
    pred = torch.argmax(logits).item()
    print(f'Model prediction: {pred}')

image_path = "pixil-7-demo.png" #@param {type:"string"}
#@markdown 

predict(image_path)